# Uso avanzado de bases de datos NoSQL


In [6]:
from pymongo import MongoClient

# Conexión al servidor local
client = MongoClient('localhost', 27017)

# Selección de la base de datos
db = client['localesMadrid']
# Selección de la colección
collection = db['madridDbLocalesB']

#### 1.	Los locales del barrio Guindalera de Salamanca, por motivos de la desescalada, no podrán abrir y tendrán que permanecer cerrados. Cambiad la situación del local a Cerrado utilizando el id correspondiente. Recordad que debéis constatar los tipos de situaciones que existan y seguir con la misma codificación. A estos mismos locales, cambiad la situación de la terraza a cerrada siguiendo las mismas premisas anteriores.

In [7]:
filtro = {"desc_distrito_local": "SALAMANCA", 'desc_barrio_local':'GUINDALERA'}
# Actualización: cambiar estado del local (solo descripción) y de la terraza (id = 8)
nuevos_valores = {
    "$set": {
        "desc_situacion_local": "Cerrado",
        "id_situacion_terraza": 8,        
        "desc_situacion_terraza": "Cerrada"
    }
}

resultado = collection.update_many(filtro, nuevos_valores)

cursor = collection.find(filtro).limit(5)

for doc in cursor:
    print(doc)

{'_id': 33, 'id_local': 270403150, 'id_distrito_local': 4, 'desc_distrito_local': 'SALAMANCA', 'desc_barrio_local': 'GUINDALERA', 'clase_vial_edificio': 'AVENIDA', 'num_edificio': 2, 'Cod_Postal': 28028, 'coordenada_x_local': '442586,58', 'coordenada_y_local': '4476524,51', 'desc_situacion_local': 'Cerrado', 'Nombre': 'HONTANARES', 'id_periodo_terraza': 1, 'desc_periodo_terraza': 'Anual', 'id_situacion_terraza': 8, 'desc_situacion_terraza': 'Cerrada', 'Superficie_ES': '21,06', 'DESC_CLASE': 'AVENIDA', 'DESC_NOMBRE': 'AMERICA', 'num_terraza': 2, 'cal_terraza': '', 'desc_ubicacion_terraza': 'Acera', 'hora_ini_LJ_es': '10:00:00', 'hora_fin_LJ_es': '1:00:00', 'hora_ini_LJ_ra': '10:00:00', 'hora_fin_LJ_ra': '0:00:00', 'hora_ini_VS_es': '10:00:00', 'hora_fin_VS_es': '2:30:00', 'hora_ini_VS_ra': '10:00:00', 'hora_fin_VS_ra': '0:00:00', 'mesas_aux_es': 0, 'mesas_aux_ra': 0, 'mesas_es': 9, 'mesas_ra': 9, 'sillas_es': 27, 'sillas_ra': ''}
{'_id': 34, 'id_local': 270078567, 'id_distrito_local': 4


2.	A todas las terrazas que se ubiquen en la acera, añadid un campo llamado inspeccionar y estableced el siguiente valor:
Si dispone de más de 10 mesas, true.
Si dispone de menos de 10 mesas, false.


In [ ]:
filtro_2 = {'desc_ubicacion_terraza': 'Acera'}

update_2 = [
    {
        "$set": {
            "inspeccionar": {
                "$cond": {
                    "if": {"$gt": ["$mesas_es", 10]},  # si mesas_es > 10
                    "then": 'True',                # entonces
                    "else": "False"              # sino
                }
            }
        }
    }
]

# Aplicar update_many
resultado = collection.update_many(filtro_2, update_2)
print("Documentos modificados:", resultado.modified_count)




Documentos modificados: 2319


In [9]:
cursor = collection.find(filtro_2, {"_id": 1, "mesas_es": 1, "inspeccionar": 1})

for doc in cursor:
    print(doc)

{'_id': 7, 'mesas_es': 5, 'inspeccionar': 'False'}
{'_id': 32, 'mesas_es': 10, 'inspeccionar': 'False'}
{'_id': 33, 'mesas_es': 9, 'inspeccionar': 'False'}
{'_id': 40, 'mesas_es': 8, 'inspeccionar': 'False'}
{'_id': 41, 'mesas_es': 6, 'inspeccionar': 'False'}
{'_id': 44, 'mesas_es': 6, 'inspeccionar': 'False'}
{'_id': 51, 'mesas_es': 14, 'inspeccionar': 'True'}
{'_id': 52, 'mesas_es': 3, 'inspeccionar': 'False'}
{'_id': 54, 'mesas_es': 6, 'inspeccionar': 'False'}
{'_id': 56, 'mesas_es': 4, 'inspeccionar': 'False'}
{'_id': 58, 'mesas_es': 8, 'inspeccionar': 'False'}
{'_id': 59, 'mesas_es': 10, 'inspeccionar': 'False'}
{'_id': 60, 'mesas_es': 8, 'inspeccionar': 'False'}
{'_id': 63, 'mesas_es': 3, 'inspeccionar': 'False'}
{'_id': 66, 'mesas_es': 4, 'inspeccionar': 'False'}
{'_id': 67, 'mesas_es': 12, 'inspeccionar': 'True'}
{'_id': 68, 'mesas_es': 5, 'inspeccionar': 'False'}
{'_id': 69, 'mesas_es': 8, 'inspeccionar': 'False'}
{'_id': 72, 'mesas_es': 9, 'inspeccionar': 'False'}
{'_id': 73,

3.	A las terrazas que se deban inspeccionar, añadid 2 mesas más auxiliares y 8 sillas más disponibles (tanto en período estacional como el resto del año).